# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1158, -0.0945, -0.0713,  0.0324, -0.0321,  0.1159, -0.1909,  0.0924,
         -0.0214,  0.1532],
        [-0.0409, -0.0958,  0.0155,  0.0769,  0.0556,  0.0178, -0.1374,  0.0604,
         -0.0519,  0.1232]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-0.2473,  0.0613,  0.0079, -0.0680, -0.1992, -0.1263, -0.2046,  0.2680,
         -0.0224,  0.0282],
        [-0.1782,  0.0269,  0.0257, -0.0222, -0.1739, -0.0795, -0.1392,  0.2031,
          0.1165,  0.0724]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.3068, -0.0935,  0.1136, -0.2365,  0.1202,  0.0408,  0.0850,  0.1886,
         -0.1318,  0.1303],
        [ 0.2005, -0.0967,  0.2273, -0.0845,  0.1270, -0.0316,  0.0890,  0.1080,
         -0.0620,  0.1010]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2784, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1057, grad_fn=<SumBackward0>)